In [5]:
# @title Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
# Then read the parquet file
df_h = pd.read_parquet('../data/futures_data_local/FF/FFIH4/FFIH4.parquet')
# for more info of this instrument nomenclature https://developers.lseg.com/en/article-catalog/article/reconstructing-rics-for-expired-futures-contracts

In [13]:
#read all files of ftse
df_o = pd.DataFrame()
for file in os.listdir('../data/FTSE_2024_data_parquet'):
    if file.endswith('.parquet'):
        df_temp = pd.read_parquet(os.path.join('../data/FTSE_2024_data_parquet', file))
        #filter for same RIC as in hedge data
        df_temp = df_temp[df_temp['Alias Underlying RIC'] == 'FFIH4']
        df_o = pd.concat([df_o, df_temp], ignore_index=True)


In [ ]:
# @title Data Input
#df_h = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_hedge_data.csv',low_memory=False)
#df_o = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/order_book_FTSE.csv')

In [22]:
# @title Function definitions
#supporter function definitions
def cumulative_sum_by_key(df, lob_state_depth):
    """
    Calculates the cumulative sum across a sequence of columns in a Pandas DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        lob_state_depth (int): The number of keys to include (e.g., 10 for L1 to L10).

    Returns:
        pd.DataFrame: A new DataFrame with the cumulative sum columns,
                          or None if the input DataFrame is invalid or
                          the required keys are not found.  The original DataFrame
                          is not modified.
    """
    if not isinstance(df, pd.DataFrame):
        print("Error: Input must be a Pandas DataFrame.")
        return None

    if not isinstance(lob_state_depth, int) or lob_state_depth <= 0:
        print("Error: num_keys must be a positive integer.")
        return None

    cumulative_sum_data = {}
    for i in range(1, lob_state_depth + 1):
        akey = f'L{i}-AskSize'
        bkey = f'L{i}-BidSize'
        cumulative_akey = f'L{i}-AskVolume'
        cumulative_bkey = f'L{i}-BidVolume'
        if akey not in df.columns or bkey not in df.columns:
            print(f"Error: Key '{akey}' or {bkey} not found in DataFrame columns.")
            return None

        if i == 1:
            cumulative_sum_data[cumulative_akey] = df[akey]  # First cumsum is just the first value
            cumulative_sum_data[cumulative_bkey] = df[bkey]  # First cumsum is just the first value
        else:
            cumulative_sum_data[cumulative_akey] = df[akey] + cumulative_sum_data[f'L{i-1}-AskVolume']
            cumulative_sum_data[cumulative_bkey] = df[bkey] + cumulative_sum_data[f'L{i-1}-BidVolume']

    return pd.DataFrame(cumulative_sum_data)



def filter_hedge_data(df_h, ric,verbose):
  if 'TRANSACTTIME' not in df_h.columns:
      print('cannot run on already filtered data')
      return df_h
  df_h = df_h[['CLORDID', 'SIDE', 'ORDERQTY', 'PRICE', 'CURRENCY', 'TIMEINFORCE', 'MKT_PRICE', 'BID', 'OFFER', 'VWAP', 'STATUS', 'TRANSACTTIME', 'EXECTYPE', 'CUMQTY', 'LEAVESQTY', 'EXEC_PRICE', 'RIC']]
  df_h = df_h[df_h['RIC'] == ric]
  df_h = df_h[~((df_h['ORDERQTY'] == 1) & (df_h['EXECTYPE'] == 4))]
  df_h = df_h[~((df_h['CUMQTY']==0) & (df_h['LEAVESQTY']==0))]
  #remove the duplicates keys is exactly equal while also making sure that CUMQTY and all other keys are also equal
  df_h = df_h.drop_duplicates(subset=['CLORDID', 'SIDE', 'ORDERQTY', 'PRICE', 'CURRENCY', 'TIMEINFORCE', 'MKT_PRICE', 'BID', 'OFFER', 'VWAP', 'STATUS', 'EXECTYPE', 'CUMQTY', 'LEAVESQTY', 'EXEC_PRICE', 'RIC'], keep='first')

  df_h = df_h.rename(columns={'TRANSACTTIME': 'hedge_time'})
  if(verbose): print('pd.timestamp:', type(df_h['hedge_time'].iloc[0]) is pd.Timestamp)
  if(verbose): print('np.datetime64:',type(df_h['hedge_time'].iloc[0]) is np.datetime64)
  if(verbose): print('str:',type(df_o['hedge_time']) is str)
  #convert to pd.timestamp
  df_h['hedge_time'] = pd.to_datetime(df_h['hedge_time'])
  if(verbose): print(type(df_h['hedge_time'].iloc[0]) is pd.Timestamp)
  df_h['hedge_time'] = df_h['hedge_time'].dt.tz_localize(None) #remove timezone from df_h['TRANSACTTIME']
  df_h = df_h.copy().sort_values(by=['hedge_time', 'CUMQTY'])

  if not df_h['hedge_time'].is_monotonic_increasing:
    #instead of error, just give me a warning
    print("Warning: df_h['hedge_time'] is not sorted correctly! But not needed for index matching ")
    #raise ValueError("df_h['hedge_time'] is not sorted correctly! Add sort_values()")
  return df_h

def filter_lob_data(df_o, ric,verbose):
  if 'Date-Time' not in df_o.columns:
      print('cannot run on already filtered data')
      return df_o
  #sort ascending by Date-Time
  df_o = df_o.sort_values(by=['Date-Time'])
  df_o = df_o[df_o['Alias Underlying RIC'] == ric]
  df_o = df_o.rename(columns={'Date-Time': 'lob_time'})
  if(verbose): print('pd.timestamp:', type(df_o['lob_time'].iloc[0]) is pd.Timestamp)
  if(verbose): print('np.datetime64:',type(df_o['lob_time'].iloc[0]) is np.datetime64)
  if(verbose): print('str:',type(df_o['lob_time']) is str)
  #convert to pd.timestamp
  df_o['lob_time'] = pd.to_datetime(df_o['lob_time'])
  df_o['lob_time'] = df_o['lob_time'].dt.tz_localize(None) #remove timezone from df_o['lob_time']
  #df_o= df_o.copy().sort_values(by=['lob_time'])
  if not df_o['lob_time'].is_monotonic_increasing:
    raise ValueError("df_o['lob_time'] is not sorted correctly! Add sort_values()")
  return df_o

def add_public_features(df_o,verbose):
  if(verbose):print('Make sure df are sorted!')
  market_spread = df_o['L1-AskPrice'] - df_o['L1-BidPrice']
  if(verbose): print(market_spread)
  mid_price = (df_o['L1-AskPrice'] + df_o['L1-BidPrice'])/2
  if(verbose): print(mid_price)
  #create a pandas data frame of volume imbalance considering each level, for eg. for l2 get ratio of volume only upto level 2
  bbo_imbalance= market_spread / mid_price / 2
  if(verbose): print(bbo_imbalance)
  volume_imbalance=pd.DataFrame()
  cum_volume= cumulative_sum_by_key(df_o, 10)
  for i in range(1, 11):
      volume_imbalance[f'L{i}-VolumeImbalance'] = cum_volume[f'L{i}-AskVolume'] / cum_volume[f'L{i}-BidVolume']
  if(verbose): print(volume_imbalance.head())

  df_o_diff = df_o.filter(regex='-Ask|-Bid').diff().fillna(0)
  df_o_diff = df_o_diff.rename(columns={col: col + '-diff' for col in df_o_diff.columns})
  if(verbose): print(df_o_diff.head())


  #combine all results into single dataframe
  df_public_features = pd.DataFrame({
        'MarketSpread': market_spread,
        'MidPrice': mid_price,
        'BBOImbalance': bbo_imbalance,
    })
  df_public_features = pd.concat([df_o, df_public_features, volume_imbalance, df_o_diff], axis=1)
  if(verbose): print(df_public_features.head())

  return df_public_features

def plot_lob_state(df_o,irow,depth):
  depth=10
  #irow=0

  plt.figure(figsize=(10, 5))
  max_ask_price= df_o[[f'L{i}-AskPrice' for i in range(1, depth+1)]].iloc[irow].max()
  min_ask_price= df_o[[f'L{i}-AskPrice' for i in range(1, depth+1)]].iloc[irow].min()
  max_bid_price= df_o[[f'L{i}-BidPrice' for i in range(1, depth+1)]].iloc[irow].max()
  min_bid_price= df_o[[f'L{i}-BidPrice' for i in range(1, depth+1)]].iloc[irow].min()

  plt.xticks(np.arange(min_bid_price, max_ask_price, step=0.5))
  plt.xticks(rotation=90)
  plt.gca().xaxis.set_major_formatter('{:.1f}'.format)
  for i in range(1, 11):  # Loop through levels L1 to L10
      plt.bar(df_o[f'L{i}-AskPrice'].iloc[irow], df_o[f'L{i}-AskSize'].iloc[irow],
              label=f'L{i}', width=0.5, edgecolor=(0.1, 0.2, 0.9, 0.1),
              linewidth=0.5, color=(0.1, 0.2, 0.5, 0.1), hatch='xxxx')
      plt.bar(df_o[f'L{i}-BidPrice'].iloc[irow], -df_o[f'L{i}-BidSize'].iloc[irow],
              label=f'L{i}', width=0.5, edgecolor=(0.9, 0.2, 0.1, 0.1),
              linewidth=0.5, color=(0.9, 0.2, 0.1, 0.1), hatch='xxxx')
  plt.text(max_ask_price, 0, 'Mid price:{:.2f}'.format((min_ask_price + max_bid_price)/2), ha='right', va='top')
  plt.text(max_ask_price, -1, 'Spread: {:.2f}'.format(min_ask_price - max_bid_price), ha='right', va='top')
  plt.axvspan(max_bid_price, min_ask_price, alpha=0.1, color='green')
  plt.xlabel('Price')
  plt.ylabel('Size')
  plt.show()

def find_closest_lob_indexes(df_h, df_o, hedge_irow, t_threshold):
    hedge_time = df_h[ 'hedge_time'].iloc[hedge_irow]
    lower_bound = hedge_time - pd.Timedelta(seconds=t_threshold)
    upper_bound = hedge_time + pd.Timedelta(seconds=t_threshold)
    #closest_df = df_o[(df_o['lob_time'] >= lower_bound) & (df_o['lob_time'] <= upper_bound)] #slower but simpler execution
    start_idx= df_o['lob_time'].searchsorted(lower_bound, side='left')
    end_idx= df_o['lob_time'].searchsorted(upper_bound, side='right')
    if end_idx-start_idx>4: print("Warning: time window seems too big,or too many trades in small window, possible error in matching")
    return start_idx, end_idx#,closest_df


#side 1 seems ask most of the time
#but when spread is small, side might not match exactly


In [17]:
df_h=filter_hedge_data(df_h,'FFIH4', False)
#df_h['ORDERQTY'].value_counts()


In [28]:
df_h[df_h['EXECTYPE'] == 4]['LEAVESQTY'].value_counts()
df_h_sorted = df_h.sort_values(by=['hedge_time'])

In [23]:
df_o=filter_lob_data(df_o,'FFIH4', False)
df_o

,#RIC,Alias Underlying RIC,Domain,lob_time,GMT Offset,Type,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
218129,FFIc1,FFIH4,Market Price,2024-01-02 01:00:45.363475304,0,Normalized LL2,7754.0,8.0,7754.5,2.0,...,7758.0,3.0,7750.0,5.0,7758.5,6.0,7749.5,5.0,7759.0,6.0
218130,FFIc1,FFIH4,Market Price,2024-01-02 01:00:51.268531784,0,Normalized LL2,7755.5,2.0,7757.0,4.0,...,7760.5,6.0,7751.5,7.0,7761.0,6.0,7751.0,6.0,7761.5,9.0
218131,FFIc1,FFIH4,Market Price,2024-01-02 01:00:51.408858431,0,Normalized LL2,7755.5,3.0,7757.0,4.0,...,7760.5,6.0,7751.5,7.0,7761.0,6.0,7751.0,6.0,7761.5,9.0
218132,FFIc1,FFIH4,Market Price,2024-01-02 01:01:03.144219958,0,Normalized LL2,7756.0,2.0,7757.5,4.0,...,7761.0,6.0,7752.0,4.0,7761.5,8.0,7751.5,7.0,7762.0,4.0
218133,FFIc1,FFIH4,Market Price,2024-01-02 01:01:11.579159435,0,Normalized LL2,7756.0,5.0,7757.0,5.0,...,7760.5,6.0,7752.0,4.0,7761.0,6.0,7751.5,7.0,7761.5,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123177,FFIc1,FFIH4,Market Price,2024-03-14 20:50:35.363783486,0,Normalized LL2,7736.0,9.0,7737.0,1.0,...,7740.5,5.0,7732.0,4.0,7741.0,8.0,7731.5,6.0,7741.5,8.0
123178,FFIc1,FFIH4,Market Price,2024-03-14 20:53:04.154332420,0,Normalized LL2,7735.0,3.0,7736.0,6.0,...,7739.5,4.0,7731.0,4.0,7740.0,4.0,7730.5,4.0,7740.5,4.0
123179,FFIc1,FFIH4,Market Price,2024-03-14 20:57:53.139424588,0,Normalized LL2,7733.0,3.0,7734.5,3.0,...,7738.5,3.0,7729.0,4.0,7739.0,3.0,7728.5,3.0,7739.5,4.0
123180,FFIc1,FFIH4,Market Price,2024-03-14 20:59:00.338586636,0,Normalized LL2,7736.0,19.0,7738.0,9.0,...,7750.0,1.0,7719.0,2.0,7751.0,2.0,7718.0,2.0,7755.0,1.0


In [24]:

irow=121
s,e = find_closest_lob_indexes(df_h, df_o, irow, 2)
print(s,e)
df_h['EXEC_PRICE'].iloc[irow]

713 713


np.float64(7772.0)

In [29]:
#loop or check if the given value matches over list of keys with regex
depth=10
if df_h['SIDE'].iloc[irow]==1 :
  key_regex = '-Ask'
  key_side = +1
else:
  key_regex = '-Bid'
  key_side = -1

ord_size=1
if df_h['ORDERQTY'].iloc[irow]!=1 & df_h['CUMQTY'].iloc[irow]!=1:
  ord_size= df_h['CUMQTY'].iloc[irow]-df_h['CUMQTY'].iloc[irow]


for i in range(1, depth+1):  # Loop through levels L1 to L10
  if df_o[f'L{i}{key_regex}Price'].iloc[s:e+1].eq(df_h_sorted['EXEC_PRICE'].iloc[irow]).any():
    #get the index where it is true and not idxmax
    idx = df_o[f'L{i}{key_regex}Price'].iloc[s:e+1].eq(df_h_sorted['EXEC_PRICE'].iloc[irow]).idxmax()
    print('first price match is at',idx, 'in lob')
    if ord_size <= df_o[f'L{i}{key_regex}Size'].iloc[idx]:
      df_o.loc[idx, 'hedge found'] = True
      df_o.loc[idx, 'hedge level'] = key_side*i
      print('hedge found', key_side*i)
    else:
      df_o.loc[idx, 'hedge found'] = False
      print('price found, size mismatch')



for key in df_o.columns:
    if key_regex in key:
        if df_o[key].iloc[s:e+1].eq(df_h_sorted['EXEC_PRICE'].iloc[irow]).any():
            print('match at', key)
            #extract L{i} i from the key
            level = int(key.split('-')[0][1:])
            print('level is ',level)



first price match is at 218842 in lob
hedge found 1
match at L1-AskPrice
level is  1


In [30]:
def match_hedges(df_h, df_o, t_threshold):
    #df_o['hedge found'] = False
    #df_o['hedge level'] = 0
    n_found=0
    n_mismatch=0
    for irow in range(len(df_h)):
        s, e = find_closest_lob_indexes(df_h, df_o, irow, t_threshold)
        if df_h['SIDE'].iloc[irow]==1 :
          key_regex = '-Ask'
          key_side = +1
        else:
          key_regex = '-Bid'
          key_side = -1
        ord_size=1
        if df_h['ORDERQTY'].iloc[irow]!=1 & df_h['CUMQTY'].iloc[irow]!=1:
          if df_h['CLORDID'].iloc[irow]==df_h['CLORDID'].iloc[irow-1]:
            ord_size= df_h['CUMQTY'].iloc[irow]-df_h['CUMQTY'].iloc[irow-1]

        for key in df_o.columns:
          if key_regex in key:
            if df_o[key].iloc[s:e+1].eq(df_h_sorted['EXEC_PRICE'].iloc[irow]).any():
              idx = df_o[key].iloc[s:e+1].eq(df_h_sorted['EXEC_PRICE'].iloc[irow]).idxmax()
              if ord_size <= df_o[key.replace('Price','Size')].iloc[idx]:
                level = int(key.split('-')[0][1:])
                df_o.loc[idx, 'hedge found'] = True
                df_o.loc[idx, 'hedge level'] = key_side*level
                df_o.loc[idx, 'hedge size'] = ord_size
                df_o.loc[idx, 'hedge id'] = df_h['CLORDID'].iloc[irow]
                df_o.loc[idx, 'hedge time'] = df_h['hedge_time'].iloc[irow]
                n_found=n_found+1
                break
            #else:
            # a function that matches with +-0.5 price
            # at L1 the sides can switch
    print('found',1.0*n_found/len(df_h),'% exact matches')
    return df_o


In [31]:
df_o= match_hedges(df_h, df_o, 2)

found 0.5124861629674102 % exact matches


In [36]:
#print the df_o where 'hedge found' is True
df_o[df_o['hedge found'] == True]
count = df_o[df_o['hedge found'] == True].shape[0]

In [33]:
df_o['hedge size'].value_counts()

hedge size
1.0     12849
2.0        54
3.0        33
4.0        23
0.0        14
5.0         9
6.0         5
9.0         4
7.0         4
11.0        3
8.0         2
45.0        1
16.0        1
10.0        1
Name: count, dtype: int64

In [37]:
# % of hedge orders matched 
perc = count/df_h_sorted.shape[0]
print(f"Percentage of hedge orders matched: {perc:.2f}%")

Percentage of hedge orders matched: 0.24%
